## FNN
Fuzzy neural networks

In [2]:
import torch.nn.functional as F
from torch import nn
import torch
import math
import csv
from fcmeans import FCM
import numpy as np

## read data
dataset preprocess: Rescaling (min-max normalization、range scaling)

https://cloud.tencent.com/developer/article/1803086

In [3]:
# map range: [a, b]
def rescaling(x, a, b):
    features = [i[0] for i in x]
    features = np.array(features)
    col_max = features.max(axis=0).tolist() # max of col
    col_min = features.min(axis=0).tolist() # min of col
    data_num = len(x)
    feature_num = len(features[0])
    for i in range(data_num):
        for j in range(feature_num):
            val = x[i][0][j]
            val = a + (val - col_min[j])*(b-a) /(col_max[j] - col_min[j])
            x[i][0][j] = val
    return x
    
def read_data():
    with open('adult.tsv', 'r', encoding='utf-8') as file_obj:
        lines_obj = csv.reader(file_obj,delimiter='\t')
        line_list =[]
        for line in lines_obj:
            if line[0] != 'age':
                for i in range(len(line)):
                    line[i] = float(line[i])
                line[-1] = int(line[-1])
            features = line[0:-2]
            target = line[-1]
            line_list.append([features, target])
        head = line_list[0]
        train_data = line_list[1:-50]
        train_len = len(train_data)
        test_data = line_list[-50:]
        test_len = len(test_data)
        train_data = rescaling(train_data,0,1)
        test_data = rescaling(test_data,0,1)
    return head,train_data, train_len, test_data, test_len
head,train_data, train_len, test_data, test_len= read_data()
print("train: ", train_len, "test_data: ", test_len)
print("head: ",head)
for d in train_data[0:50]:
    print("data :", d)

train:  48792 test_data:  50
head:  [['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week'], 'target']
data : [[0.3013698630136986, 0.875, 0.044131207652990466, 0.6, 0.8, 0.6666666666666666, 0.07142857142857142, 0.2, 1.0, 1.0, 0.021740217402174022, 0.0, 0.3979591836734694], 1]
data : [[0.4520547945205479, 0.75, 0.048051741576264365, 0.6, 0.8, 0.3333333333333333, 0.2857142857142857, 0.0, 1.0, 1.0, 0.0, 0.0, 0.12244897959183673], 1]
data : [[0.2876712328767123, 0.5, 0.13758131133233883, 0.7333333333333333, 0.5333333333333333, 0.0, 0.42857142857142855, 0.2, 1.0, 1.0, 0.0, 0.0, 0.3979591836734694], 1]
data : [[0.4931506849315068, 0.5, 0.15048626121783487, 0.06666666666666667, 0.4, 0.3333333333333333, 0.42857142857142855, 0.0, 0.5, 1.0, 0.0, 0.0, 0.3979591836734694], 1]
data : [[0.1506849315068493, 0.5, 0.2206350656072092, 0.6, 0.8, 0.3333333333333333, 0.7142857142857143, 1

## FCM
find center and sigma

In [4]:

def fcm(data, cluster_num, h):
    fcm = FCM(n_clusters=cluster_num)
    data = [i[0] for i in data]
    data = np.array(data)
    fcm.fit(data)
    centers = fcm.centers
    membership = fcm.soft_predict(data)
    feature_num = len(data[0])
    data_num = len(data)
    sigma = []
    for i in range(cluster_num):
        feature_sigma = []
        for j in range(feature_num):
            a = 0
            b = 0
            for k in  range(data_num):
                x = data[k][j]
                a = a + membership[k][i]* ((x-centers[i][j]) ** 2)
                b = b + membership[k][i]
            feature_sigma.append(h*a/b)
        sigma.append(feature_sigma)
    return torch.tensor(centers),torch.tensor(sigma)
h = 10
print(fcm(train_data, 6, h))



(tensor([[0.2534, 0.4664, 0.1179, 0.7054, 0.6037, 0.4995, 0.4401, 0.5657, 0.9012,
         0.0947, 0.0062, 0.0131, 0.3635],
        [0.3112, 0.4951, 0.1205, 0.6861, 0.6100, 0.4100, 0.4821, 0.1715, 0.9323,
         0.9006, 0.0118, 0.0216, 0.4197],
        [0.2534, 0.4664, 0.1179, 0.7054, 0.6037, 0.4995, 0.4401, 0.5657, 0.9012,
         0.0947, 0.0062, 0.0131, 0.3635],
        [0.3112, 0.4951, 0.1205, 0.6861, 0.6100, 0.4100, 0.4821, 0.1715, 0.9323,
         0.9006, 0.0118, 0.0216, 0.4197],
        [0.3112, 0.4951, 0.1205, 0.6861, 0.6100, 0.4100, 0.4821, 0.1715, 0.9323,
         0.9006, 0.0118, 0.0216, 0.4197],
        [0.3112, 0.4951, 0.1205, 0.6861, 0.6100, 0.4100, 0.4821, 0.1715, 0.9323,
         0.9006, 0.0118, 0.0216, 0.4197]], dtype=torch.float64), tensor([[0.3715, 0.3542, 0.0502, 0.7116, 0.2716, 0.8437, 0.9586, 1.1669, 0.5200,
         2.4955, 0.0483, 0.0722, 0.1610],
        [0.3433, 0.3265, 0.0514, 0.6511, 0.3025, 0.5417, 0.8939, 0.8410, 0.4176,
         1.5930, 0.0584, 0.0908, 0

## FNN model

In [5]:

class MemberShip(nn.Module):
    def __init__(self, feature_num, rule_num, center, sigma):
        super(MemberShip, self).__init__()
        self.feature_num = feature_num
        self.rule_num = rule_num
        self.center = nn.Parameter(center)
        self.sigma = nn.Parameter(sigma)
    def forward(self, x):
        mb_list = []
        for i in range(self.rule_num):
            tmp_list = []
            for j in range(self.feature_num): 
                value = ((x[j]-self.center[i][j]) / self.sigma[i][j]) ** 2
                value = torch.exp(-(value /2))
                tmp_list.append(value)
            mb_list.append(tmp_list)
        membership = torch.tensor(mb_list).float()
        return membership

class RuleGen(nn.Module):
    def __init__(self,rule_num):
        super(RuleGen, self).__init__()
        self.rule_num = rule_num
    def forward(self, membership):
        # x: membership array
        # rule_num * feature_num 
        rule_list = []
        feature_num = len(membership[0])
        rule_num = self.rule_num 
        for i in range(rule_num):
            value = 1.0
            for j in range(feature_num):
                value = value * membership[i][j]
            rule_list.append(value)
        rule = torch.tensor(rule_list).float()
        return rule

class FNN(nn.Module):
    def __init__(self,feature_num , rule_num, center,sigma ):
        super(FNN, self).__init__()
        self.rule_num = rule_num
        self.ms_layer = MemberShip(feature_num, rule_num, center,sigma )
        self.rule_layer = RuleGen(rule_num)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(rule_num,1024)
        self.fc2 = nn.Linear(1024,2)
    def norm(self, x):
        sum_x = torch.sum(x, dim=-1)
        if sum_x == 0.0:
            return x
        return x/sum_x
    def forward(self, x):
        membership = self.ms_layer(x)
        rule = self.rule_layer(membership)
        rule = self.norm(rule)
        output = self.fc(rule)
        output = self.fc2(output)
        return output

class MLP(nn.Module):
    def __init__(self,feature_num , rule_num, center,sigma ):
        super(MLP, self).__init__()
        self.fc = nn.Linear(feature_num, 2048)
        self.relu = nn.ReLU()
        self.drop_out= nn.Dropout(0.1)
        self.fc2 = nn.Linear(2048, 1024)
        self.relu2 = nn.ReLU()
        self.drop_out2= nn.Dropout(0.1)
        self.fc3 = nn.Linear(1024, 2)
    def forward(self, x):
        input = self.fc(x)
        input = self.relu(input)
        input = self.drop_out(input)
        input = self.fc2(input)
        input = self.relu2(input)
        input = self.drop_out2(input)
        output = self.fc3(input)
        return output

## train

In [6]:
def valid(model, test_data):
    count = 0
    acc = 0
    total_loss = 0
    softmax = nn.Softmax(dim=-1)
    criterion = nn.CrossEntropyLoss()
    for features, target in test_data:
        x = torch.tensor(features)
        target = torch.tensor(target)
        output = fnn(x)
        loss = criterion(output, target)
        output = softmax(output)
        predict = torch.argmax(output, dim = -1)
        count = count +1
        total_loss = total_loss +loss.item()
        if predict == target:
            acc = acc +1
    print("acc:", acc)
    return total_loss/count, acc/count
feature_num = 13
cluster_num = 100
rule_num = cluster_num
h = 1.0
center,sigma = fcm(train_data, cluster_num= cluster_num, h= h)
fnn = MLP(13, rule_num, center,sigma )
optimizer = torch.optim.Adam(fnn.parameters(), lr=0.0001, weight_decay=0)
criterion = nn.CrossEntropyLoss()
## model parameters
total_params = sum(p.numel() for p in fnn.parameters())
total_trainable_params = sum(p.numel() for p in fnn.parameters() if p.requires_grad)
print("model total parameters: %d, trainable  parameters: %d " %(total_params,total_trainable_params))
epoch = 20
for i in range(epoch):
    count = 0
    total_acc= 0
    total_bleu=0
    total_loss = 0
    for features,target in train_data:
        x = torch.tensor(features)
        target = torch.tensor(target)
        output = fnn(x)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count = count+ 1
        total_loss =total_loss + loss.item()
        if count %100 ==0:
            test_loss, acc = valid(fnn, test_data)
            print("epoch: ", i, "count: ", count, ", train loss: ", total_loss/count, ", test loss: ", test_loss, ",acc: ",acc)
            # print("epoch: ", i,", count: ", count, ", train loss: ", total_loss/count)
        

model total parameters: 2128898, trainable  parameters: 2128898 
acc: 38
epoch:  0 count:  100 , train loss:  0.9438722412951757 , test loss:  0.522284744232893 ,acc:  0.76
acc: 38
epoch:  0 count:  200 , train loss:  0.7218370665289694 , test loss:  0.512912072353065 ,acc:  0.76
acc: 38
epoch:  0 count:  300 , train loss:  0.6708282938075718 , test loss:  0.49017260571941734 ,acc:  0.76
acc: 38
epoch:  0 count:  400 , train loss:  0.63399121685914 , test loss:  0.48561472378671167 ,acc:  0.76
acc: 38
epoch:  0 count:  500 , train loss:  0.5932356092568952 , test loss:  0.6176979295921047 ,acc:  0.76
acc: 39
epoch:  0 count:  600 , train loss:  0.5758421517843332 , test loss:  0.47903116662055256 ,acc:  0.78
acc: 42
epoch:  0 count:  700 , train loss:  0.5526620390155378 , test loss:  0.45427639373927375 ,acc:  0.84
acc: 38
epoch:  0 count:  800 , train loss:  0.5434223925130754 , test loss:  0.40144343370979185 ,acc:  0.76
acc: 43
epoch:  0 count:  900 , train loss:  0.534136508483902

KeyboardInterrupt: 